```py
EncoderDecoder.forward 输入: [torch.Size([16, 128]), torch.Size([16, 128]), torch.Size([16, 1, 1, 128])] # NT: input_ids [16, 128]
    # HL: Inputs 模块的嵌入层 + 位置编码
    # NOTE: 16是批次大小，16个句子，每个句子128个tokens
    Sequential.forward 输入: [torch.Size([16, 128])] 
        Embeddings.forward 输入: [torch.Size([16, 128])]
            Embedding.forward 输入:[torch.Size([16, 128])] 输出:torch.Size([16, 128, 768])
        Embeddings.forward 输出: torch.Size([16, 128, 768])
        PositionalEncoding.forward 输入: [torch.Size([16, 128, 768])]
            Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
        PositionalEncoding.forward 输出: torch.Size([16, 128, 768])
    Sequential.forward 输出: torch.Size([16, 128, 768])
    # HL: Encoder 模块 (input_embeddings, src_mask)
    # NT: input_embeddings [batch_size, seq_len, embedding_dim] # 每一个 token -> 768 维的向量
    # NT: src_mask [batch_size, 1, 1, seq_len]: 0 表示[padding]
    Encoder.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
        EncoderLayer.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
            # NT: 原始论文(Post-LN): Output = LayerNorm(Sublayer(Input) + Input)
            # NT: 实践最优(Pre-LN): Output = Sublayer(LayerNorm(Input)) + Input
            #                CODE: x + self.dropout(sublayer(self.norm(x)))
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])] # HL: 多头注意力层(self_attn(x, x, x, src_mask))
                # SOL: 所以这里先层归一化再进行计算
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Dropout.forward 输入:[torch.Size([16, 8, 128, 128])] 输出:torch.Size([16, 8, 128, 128])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输出: torch.Size([16, 128, 768])
                Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
            SublayerConnection.forward 输出: torch.Size([16, 128, 768])
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])] # HL: 前馈层
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                PositionwiseFeedForward.forward 输入: [torch.Size([16, 128, 768])]
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 2048])
                    Dropout.forward 输入:[torch.Size([16, 128, 2048])] 输出:torch.Size([16, 128, 2048])
                    Linear.forward 输入:[torch.Size([16, 128, 2048])] 输出:torch.Size([16, 128, 768])
                PositionwiseFeedForward.forward 输出: torch.Size([16, 128, 768])
                Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
            SublayerConnection.forward 输出: torch.Size([16, 128, 768])
        EncoderLayer.forward 输出: torch.Size([16, 128, 768]) * 6 # HL: 6个EncoderLayer堆叠，6次结束之后的K和V才会传递到Decoder的Encoder-Decoder Attention层
        LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
    Encoder.forward 输出: torch.Size([16, 128, 768])
    Sequential.forward 输入: [torch.Size([16, 128])] # HL: Outputs 模块的嵌入层+位置编码
        Embeddings.forward 输入: [torch.Size([16, 128])]
            Embedding.forward 输入:[torch.Size([16, 128])] 输出:torch.Size([16, 128, 768])
        Embeddings.forward 输出: torch.Size([16, 128, 768])
        PositionalEncoding.forward 输入: [torch.Size([16, 128, 768])]
            Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
        PositionalEncoding.forward 输出: torch.Size([16, 128, 768])
    Sequential.forward 输出: torch.Size([16, 128, 768])
    # HL: Decoder 模块 (output_embedding, encoder_output, tgt_mask)
    # NT: 第二个输入来自encoder_output
    Decoder.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
        # HL: 解码器的Decoder的输入(
            # HL: 目标序列的嵌入表示[batch_size, tgt_seq_len, d_model] + 
            # HL: 编码器encoder的输出[batch_size, input_seq_len, d_model] + 
            # HL: 目标序列的mask [batch_size, 1, 1, tgt_seq_len]
        # HL: )
        # HL: 一个Decoder层有6个一模一样的DecoderLayer子层
        DecoderLayer.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])] # HL: 多头自注意力层 (self_attn(x, x, x, tgt_mask=None))
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 128, 768])]
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Dropout.forward 输入:[torch.Size([16, 8, 128, 128])] 输出:torch.Size([16, 8, 128, 128])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输出: torch.Size([16, 128, 768])
                Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
            SublayerConnection.forward 输出: torch.Size([16, 128, 768])
            # SOL: Encoder-Decoder Attention(这个注意力层的输入包含来自Encoder的输出)
            # NT: x decoder当前状态
            # NT: memory 来自encoder的K
            # NT: memory 来自encoder的V
            # NT: src_mask 来自encoder的mask
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])] # HL: 多头注意力层(src_attn(x, memory, memory, src_mask))
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                    Dropout.forward 输入:[torch.Size([16, 8, 128, 128])] 输出:torch.Size([16, 8, 128, 128])
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeadedAttention.forward 输出: torch.Size([16, 128, 768])
                Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
            SublayerConnection.forward 输出: torch.Size([16, 128, 768])
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])] # HL: 前馈层
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                PositionwiseFeedForward.forward 输入: [torch.Size([16, 128, 768])]
                    Linear.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 2048])
                    Dropout.forward 输入:[torch.Size([16, 128, 2048])] 输出:torch.Size([16, 128, 2048])
                    Linear.forward 输入:[torch.Size([16, 128, 2048])] 输出:torch.Size([16, 128, 768])
                PositionwiseFeedForward.forward 输出: torch.Size([16, 128, 768])
                Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
            SublayerConnection.forward 输出: torch.Size([16, 128, 768])
        DecoderLayer.forward 输出: torch.Size([16, 128, 768]) * 6
        LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
    Decoder.forward 输出: torch.Size([16, 128, 768])
EncoderDecoder.forward 输出: torch.Size([16, 128, 768])
```

### 残差连接

In [ ]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        # 残差连接: x + dropout(sublayer(layernorm(x)))
        return x + self.dropout(sublayer(self.norm(x)))

- 残差连接:
  1. 缓解梯度消失：让梯度直接回传
     - $y=F(x)$, 导数$d\text{loss}/dx = d\text{loss}/dy \times dF(x)/dx$
     - $y=x + F(x)$, 导数$d\text{loss}/dx = d\text{loss}/dy \times (1 + dF(x)/dx)$
     - 即使$dF(x)/dx$趋于0, 仍然保证$d\text{loss}/dy$这部分
     - 让100层网络的训练像10层网络一样稳定
     - 底层参数也能获得有效的梯度更新
  2. 保持信息流动：保留原始输入信息
     - 每层输出 = 原始输入 + 学习到的变化, output = input + learned_change
     - 恒等映射捷径：原始输入 x 直接传递到输出
     - 选择性学习：网络只需要学习相对于输入的"残差" F(x)
     - 信息无损传递：重要特征不会在深层中丢失
  3. 训练稳定性：使深层网络更容易训练
     - 深层网络容易出现的问题:
       - Loss震荡剧烈
       - 需要精心调参和学习率策略
       - 对初始化敏感
     - 训练过程更加稳定和平滑
     - 对超参数的敏感性降低
     - 可以使用更大的学习率加速收敛